# House Prices: Advanced Regression Techniques

In [154]:
from tpot import TPOTRegressor
import pandas as pd
import numpy as nop
import tensorflow as tf
from datetime import datetime

In [155]:
train = pd.read_csv('./data/train.csv')

In [156]:
test = pd.read_csv('./data/test.csv')

In [157]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [158]:
train.shape

(1460, 81)

In [159]:
train.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 81, dtype: object

In [160]:
# Attributes which are mostly Null
mostly_nan = {x:y for x,y in dict(train.isnull().sum()).items() if y > 1200}
mostly_nan

{'Alley': 1369, 'PoolQC': 1453, 'MiscFeature': 1406}

In [161]:
train = train.drop(mostly_nan.keys(), axis = 1)

In [162]:
have_nan = {x:y for x,y in dict(train.isnull().sum()).items() if y > 0}
have_nan

{'LotFrontage': 259,
 'MasVnrType': 8,
 'MasVnrArea': 8,
 'BsmtQual': 37,
 'BsmtCond': 37,
 'BsmtExposure': 38,
 'BsmtFinType1': 37,
 'BsmtFinType2': 38,
 'Electrical': 1,
 'FireplaceQu': 690,
 'GarageType': 81,
 'GarageYrBlt': 81,
 'GarageFinish': 81,
 'GarageQual': 81,
 'GarageCond': 81,
 'Fence': 1179}

In [163]:
train['Electrical'].value_counts().index[0]

'SBrkr'

In [164]:
train[train['Electrical'].isnull()]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,Fence,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1379,1380,80,RL,73.0,9735,Pave,Reg,Lvl,AllPub,Inside,Gtl,Timber,Norm,Norm,1Fam,SLvl,5,5,2006,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,384,384,GasA,Gd,Y,NaN,754,640,0,1394,0,0,2,1,3,1,Gd,7,Typ,0,NaN,BuiltIn,2007.0,Fin,2,400,TA,TA,Y,100,0,0,0,0,0,NaN,0,5,2008,WD,Normal,167500


In [165]:
# Replace Nan with mode
for col,_ in have_nan.items():
    if train.dtypes[col] == 'object':
        mode = train[col].value_counts().index[0]
        train[col] = train[col].replace(np.nan, mode)
    else:
        median = train[col].median()
        train[col] = train[col].replace(np.nan, 0)

In [166]:
{x:y for x,y in dict(train.isnull().sum()).items() if y > 0}

{}

In [167]:
categoricals = list(train.select_dtypes(include=['object']).columns)

In [168]:
from sklearn.preprocessing import LabelBinarizer
encoders = {}
to_drop = []
for col in categoricals:
    print(f'{col} = {train[col].unique()}')
    lb = LabelBinarizer()
    encoded = lb.fit_transform(train[col])
#     print(y.shape)
    if encoded.shape[1] > 1:
        ne = pd.DataFrame(encoded, columns=[f'{col}_{clas}' for clas in lb.classes_])
        to_drop.append(col)
    else:
        ne = pd.DataFrame(encoded, columns=[col])
    train = train.drop([col], axis = 1)
    train = pd.concat([train, ne], axis=1)
    encoders[col] = lb

MSZoning = ['RL' 'RM' 'C (all)' 'FV' 'RH']
Street = ['Pave' 'Grvl']
LotShape = ['Reg' 'IR1' 'IR2' 'IR3']
LandContour = ['Lvl' 'Bnk' 'Low' 'HLS']
Utilities = ['AllPub' 'NoSeWa']
LotConfig = ['Inside' 'FR2' 'Corner' 'CulDSac' 'FR3']
LandSlope = ['Gtl' 'Mod' 'Sev']
Neighborhood = ['CollgCr' 'Veenker' 'Crawfor' 'NoRidge' 'Mitchel' 'Somerst' 'NWAmes'
 'OldTown' 'BrkSide' 'Sawyer' 'NridgHt' 'NAmes' 'SawyerW' 'IDOTRR'
 'MeadowV' 'Edwards' 'Timber' 'Gilbert' 'StoneBr' 'ClearCr' 'NPkVill'
 'Blmngtn' 'BrDale' 'SWISU' 'Blueste']
Condition1 = ['Norm' 'Feedr' 'PosN' 'Artery' 'RRAe' 'RRNn' 'RRAn' 'PosA' 'RRNe']
Condition2 = ['Norm' 'Artery' 'RRNn' 'Feedr' 'PosN' 'PosA' 'RRAn' 'RRAe']
BldgType = ['1Fam' '2fmCon' 'Duplex' 'TwnhsE' 'Twnhs']
HouseStyle = ['2Story' '1Story' '1.5Fin' '1.5Unf' 'SFoyer' 'SLvl' '2.5Unf' '2.5Fin']
RoofStyle = ['Gable' 'Hip' 'Gambrel' 'Mansard' 'Flat' 'Shed']
RoofMatl = ['CompShg' 'WdShngl' 'Metal' 'WdShake' 'Membran' 'Tar&Grv' 'Roll'
 'ClyTile']
Exterior1st = ['VinylSd' 'Meta

In [169]:
train.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,MSZoning_C (all),MSZoning_FV,MSZoning_RH,...,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2003.0,2,548,0,61,0,0,0,0,0,2,2008,208500,0,0,0,...,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,1976.0,2,460,298,0,0,0,0,0,0,5,2007,181500,0,0,0,...,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,2001.0,2,608,0,42,0,0,0,0,0,9,2008,223500,0,0,0,...,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,1998.0,3,642,0,35,272,0,0,0,0,2,2006,140000,0,0,0,...,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,2000.0,3,836,192,84,0,0,0,0,0,12,2008,250000,0,0,0,...,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [170]:
train.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,MSZoning_C (all),MSZoning_FV,MSZoning_RH,...,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2003.0,2,548,0,61,0,0,0,0,0,2,2008,208500,0,0,0,...,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,1976.0,2,460,298,0,0,0,0,0,0,5,2007,181500,0,0,0,...,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,2001.0,2,608,0,42,0,0,0,0,0,9,2008,223500,0,0,0,...,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,1998.0,3,642,0,35,272,0,0,0,0,2,2006,140000,0,0,0,...,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,2000.0,3,836,192,84,0,0,0,0,0,12,2008,250000,0,0,0,...,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [171]:
train.dtypes.value_counts()

int32      240
int64       35
float64      3
dtype: int64

In [172]:
y = train['SalePrice']
X = train
X = train.drop(['SalePrice', 'Id'], axis = 1)

In [173]:
# from sklearn.preprocessing import MinMaxScaler, StandardScaler
# scalers = {}
# for col in X:
#     scale = StandardScaler()
#     X[col] = scale.fit_transform(np.array(X[col]).reshape(-1, 1))
#     scalers[col] = scale


In [174]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [175]:
from sklearn.linear_model import LinearRegression

In [176]:
lr_model = LinearRegression()

In [177]:
lr_model.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [178]:
pred = lr_model.predict(X_test)

NotFittedError: This LinearRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [179]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [180]:
mean_squared_error(y_test, pred)

NameError: name 'y_test' is not defined

In [181]:
mean_absolute_error(y_test, pred)

NameError: name 'y_test' is not defined

In [182]:
test = test.drop(mostly_nan, axis = 1)

In [183]:
test_have_nan = {x:y for x,y in dict(test.isnull().sum()).items() if y > 0}
test_have_nan

{'MSZoning': 4,
 'LotFrontage': 227,
 'Utilities': 2,
 'Exterior1st': 1,
 'Exterior2nd': 1,
 'MasVnrType': 16,
 'MasVnrArea': 15,
 'BsmtQual': 44,
 'BsmtCond': 45,
 'BsmtExposure': 44,
 'BsmtFinType1': 42,
 'BsmtFinSF1': 1,
 'BsmtFinType2': 42,
 'BsmtFinSF2': 1,
 'BsmtUnfSF': 1,
 'TotalBsmtSF': 1,
 'BsmtFullBath': 2,
 'BsmtHalfBath': 2,
 'KitchenQual': 1,
 'Functional': 2,
 'FireplaceQu': 730,
 'GarageType': 76,
 'GarageYrBlt': 78,
 'GarageFinish': 78,
 'GarageCars': 1,
 'GarageArea': 1,
 'GarageQual': 78,
 'GarageCond': 78,
 'Fence': 1169,
 'SaleType': 1}

In [184]:
# Replace Nan with mode
for col,_ in test_have_nan.items():
    mode = test[col].value_counts().index[0]
    test[col] = test[col].replace(np.nan, mode)

In [185]:
for col in categoricals:
#     print(f'{col} = {test[col].unique()}')

    encoded = encoders[col].transform(test[col])
#     print(y.shape)
    if encoded.shape[1] > 1:
        ne = pd.DataFrame(encoded, columns=[f'{col}_{clas}' for clas in encoders[col].classes_])
        to_drop.append(col)
    else:
        ne = pd.DataFrame(encoded, columns=[col])
    test = test.drop([col], axis = 1)
    test = pd.concat([test, ne], axis=1)

In [186]:
test.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,...,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1461,20,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,270.0,882.0,896,0,0,896,0.0,0.0,1,0,2,1,5,0,1961.0,1.0,730.0,140,0,0,0,120,0,0,6,2010,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,1462,20,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,406.0,1329.0,1329,0,0,1329,0.0,0.0,1,1,3,1,6,0,1958.0,1.0,312.0,393,36,0,0,0,0,12500,6,2010,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,1463,60,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,137.0,928.0,928,701,0,1629,0.0,0.0,2,1,3,1,6,1,1997.0,2.0,482.0,212,34,0,0,0,0,0,3,2010,0,0,0,1,...,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,1464,60,78.0,9978,6,6,1998,1998,20.0,602.0,0.0,324.0,926.0,926,678,0,1604,0.0,0.0,2,1,3,1,7,1,1998.0,2.0,470.0,360,36,0,0,0,0,0,6,2010,0,0,0,1,...,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,1465,120,43.0,5005,8,5,1992,1992,0.0,263.0,0.0,1017.0,1280.0,1280,0,0,1280,0.0,0.0,2,0,2,1,5,0,1992.0,2.0,506.0,0,82,0,0,144,0,0,1,2010,0,0,0,1,...,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [187]:
Id_col = test['Id']
X_test = test.drop(['Id'], axis = 1)

In [188]:
scalers

{'MSSubClass': StandardScaler(),
 'LotFrontage': StandardScaler(),
 'LotArea': StandardScaler(),
 'OverallQual': StandardScaler(),
 'OverallCond': StandardScaler(),
 'YearBuilt': StandardScaler(),
 'YearRemodAdd': StandardScaler(),
 'MasVnrArea': StandardScaler(),
 'BsmtFinSF1': StandardScaler(),
 'BsmtFinSF2': StandardScaler(),
 'BsmtUnfSF': StandardScaler(),
 'TotalBsmtSF': StandardScaler(),
 '1stFlrSF': StandardScaler(),
 '2ndFlrSF': StandardScaler(),
 'LowQualFinSF': StandardScaler(),
 'GrLivArea': StandardScaler(),
 'BsmtFullBath': StandardScaler(),
 'BsmtHalfBath': StandardScaler(),
 'FullBath': StandardScaler(),
 'HalfBath': StandardScaler(),
 'BedroomAbvGr': StandardScaler(),
 'KitchenAbvGr': StandardScaler(),
 'TotRmsAbvGrd': StandardScaler(),
 'Fireplaces': StandardScaler(),
 'GarageYrBlt': StandardScaler(),
 'GarageCars': StandardScaler(),
 'GarageArea': StandardScaler(),
 'WoodDeckSF': StandardScaler(),
 'OpenPorchSF': StandardScaler(),
 'EnclosedPorch': StandardScaler(),
 

In [189]:
# from sklearn.preprocessing import MinMaxScaler
# for col in X_test:
#     X_test[col] = scalers[col].transform(np.array(X_test[col]).reshape(-1, 1))
     


In [190]:
tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2, random_state=96)

In [191]:
tpot.fit(X, y)


Generation 1 - Current best internal CV score: -799392827.9445789
Generation 2 - Current best internal CV score: -776869245.0050375
Generation 3 - Current best internal CV score: -776869245.0050375
Generation 4 - Current best internal CV score: -776869245.0050375
Generation 5 - Current best internal CV score: -776869245.0050375
Best pipeline: KNeighborsRegressor(RidgeCV(RobustScaler(input_matrix)), n_neighbors=40, p=1, weights=distance)


TPOTRegressor(generations=5,
              log_file=<ipykernel.iostream.OutStream object at 0x0000029CD3571C88>,
              population_size=50, random_state=96, verbosity=2)

In [192]:
my_pred = tpot.predict(X_test)
pred_series = pd.Series(my_pred, name='SalePrice')
output = pd.concat([Id_col, pred_series], axis=1)
output.to_csv(f'./submissions/tpot_kaggle_submssion_{datetime.now().strftime("%d-%b-%Y_(%H-%M-%S)")}.csv', index = False)

In [580]:
my_pred = lr_model.predict(X_test)

In [581]:
my_pred

array([109392.66174316, 152254.56469727, 186848.2947998 , ...,
       180758.60296631, 105995.24700928, 224207.48864746])

In [582]:
pred_series = pd.Series(my_pred, name='SalePrice')

In [583]:
output = pd.concat([Id_col, pred_series], axis=1)

In [584]:
output.head()

,Id,SalePrice
0,1461,109392.661743
1,1462,152254.564697
2,1463,186848.294800
3,1464,192210.252869
4,1465,205223.612793


In [592]:
output.to_csv('./submissions/lr_kaggle_submssion.csv', index = False)

In [593]:
# Simple Linear Regression got score 0.85796 and 4989 position

In [594]:
from sklearn.ensemble import RandomForestRegressor

In [595]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [596]:
rf_model = RandomForestRegressor()

In [597]:
rf_model.fit(X, y)

RandomForestRegressor()

In [598]:
Id_col = test['Id']
X_kaggle_test = test.drop(['Id'], axis = 1)

In [599]:
my_pred = rf_model.predict(X_kaggle_test)
pred_series = pd.Series(my_pred, name='SalePrice')
output = pd.concat([Id_col, pred_series], axis=1)
output.to_csv('rf_kaggle_submssion.csv', index = False)

In [600]:
# Random Forest Regressor got score 0.14668 and 2965 position

In [601]:
from sklearn.ensemble import GradientBoostingRegressor

In [602]:
gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,max_depth=1, random_state=0, loss='ls')

In [603]:
gb_model.fit(X, y)

GradientBoostingRegressor(max_depth=1, random_state=0)

In [604]:
my_pred = gb_model.predict(X_kaggle_test)
pred_series = pd.Series(my_pred, name='SalePrice')
output = pd.concat([Id_col, pred_series], axis=1)
output.to_csv('./submissions/gb_kaggle_submssion.csv', index = False)

In [605]:
 # GradientBoosting Regressor got score 0.17547

In [606]:
from sklearn.neural_network import MLPRegressor

In [607]:
mlp_model = MLPRegressor(random_state=1, max_iter=5000)

In [608]:
mlp_model.fit(X, y)

MLPRegressor(max_iter=5000, random_state=1)

In [609]:
my_pred = mlp_model.predict(X_kaggle_test)
pred_series = pd.Series(my_pred, name='SalePrice')
output = pd.concat([Id_col, pred_series], axis=1)
output.to_csv('./submissions/mlp_kaggle_submssion.csv', index = False)

In [610]:
#!pip3 install xgboost

In [226]:
import xgboost as xgb

In [227]:
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=11196)

In [228]:
xgb_model.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             random_state=11196, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [229]:
my_pred = xgb_model.predict(X_test)
pred_series = pd.Series(my_pred, name='SalePrice')
output = pd.concat([Id_col, pred_series], axis=1)
output.to_csv(f'./submissions/xgb_kaggle_submssion_{datetime.now().strftime("%d-%b-%Y_(%H-%M-%S)")}.csv', index = False)

In [399]:
# XGB Regressor got score 0.14274 and 2645 position

In [234]:
!pip install mlxtend

In [37]:
!pip install git+https://github.com/tensorflow/docs

  Cloning https://github.com/tensorflow/docs to c:\users\hassa\appdata\local\temp\pip-req-build-f9yi3uxp
  Created wheel for tensorflow-docs: filename=tensorflow_docs-0.0.0ddfe55f6903a44ad2e7a214f7578f6d349ef4a8a_-py3-none-any.whl size=111063 sha256=b03f318820f7cc78cec6ff8cab78c4d5beba0c0311ecc3b91693697fad7e372c
  Stored in directory: C:\Users\hassa\AppData\Local\Temp\pip-ephem-wheel-cache-83xa9v7f\wheels\cc\c4\d8\5341e93b6376c5c929c49469fce21155eb69cef1a4da4ce32c
Successfully built tensorflow-docs


  Running command git clone -q https://github.com/tensorflow/docs 'C:\Users\hassa\AppData\Local\Temp\pip-req-build-f9yi3uxp'


In [91]:
%load_ext tensorboard

In [127]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [138]:
def build_model():
  model = Sequential([
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(1)
  ])

  optimizer = tf.keras.optimizers.Adagrad()

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse', 'msle'])
  return model

In [139]:
x_input = tf.constant([[1,2.]], shape=(1,2))

In [140]:
model = build_model()

In [142]:
EPOCHS = 1000
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
history = model.fit(
  X, y,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)])

In [133]:
%tensorboard --logdir logs/fit

UsageError: Line magic function `%tensorboard` not found.


In [193]:
pred = model.predict(X_test).flatten()

ValueError: in user code:

    c:\users\hassa\.virtualenvs\mit_deep_learning-6vn7g6jr\lib\site-packages\tensorflow\python\keras\engine\training.py:1147 predict_function  *
        outputs = self.distribute_strategy.run(
    c:\users\hassa\.virtualenvs\mit_deep_learning-6vn7g6jr\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\hassa\.virtualenvs\mit_deep_learning-6vn7g6jr\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\hassa\.virtualenvs\mit_deep_learning-6vn7g6jr\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\users\hassa\.virtualenvs\mit_deep_learning-6vn7g6jr\lib\site-packages\tensorflow\python\keras\engine\training.py:1122 predict_step  **
        return self(x

In [144]:
pred

array([151715.77, 160436.11, 194910.3 , ..., 201772.12, 130132.82,
       221699.33], dtype=float32)

In [145]:
pred_series = pd.Series(pred, name='SalePrice')
output = pd.concat([Id_col, pred_series], axis=1)
output.to_csv(f'./submissions/tf_kaggle_submssion_{datetime.now().strftime("%d-%b-%Y_(%H-%M-%S)")}.csv', index = False)